# NLP

El objetivo de esta practica es utilizar tecnicas de NLP para mejorar la informacion obtenida a partir de la descripcion de un puesto de trabajo en Infojobs.

En este sentido, la practica tiene 2 bloques diferenciados:
- El primero, contiene el codigo necesario para realizar el Scrapping de los datos de Infojobs y volcarlos en un Dataframe Pandas
- El segundo, contiene el codigo que se ha implementado para mejorar la informacion del titulo del puesto. Se busca obtener un conjunto de palabras que sea significativo a partir del valor original y que permita hacer agrupaciones para realizar conteos (por ejemplo para un tratamiento de Clustering ulterior)

## 1. Scrapping

Librerias

In [1]:
import requests

from lxml import html,etree
from io import StringIO, BytesIO

import re
import unicodedata

import pandas as pd

Constantes

In [2]:
URL = "https://www.infojobs.net/jobsearch/search-results/list.xhtml"
ENCODING = 'ISO-8859-15'

Funciones desarrolladas

In [3]:
# Funcion para transformar los datos de Strings lxml a Strings unicode
def to_string(L):
    R = []
    for E in L:
        #R.append(E.encode('utf8','strict'))
        e=unicode(E.encode('utf8','strict').strip(),'utf-8')
        R.append(unicodedata.normalize('NFD', e).encode('ascii', 'ignore'))       

    return R
 
# Funcion para recorrer la lista de detalles que contiene el tipo de contrato, horario y salario
def parse_details(details):
    contract = []
    hours = []
    salary = []
    for i in range(0, len(details)):
        cells = details[i].getchildren()

        if len(cells) == 3:
            contract.append(cells[0].text)
            hours.append(cells[1].text)
            salary.append(cells[2].text)
        elif len(cells) == 2:
            contract.append(cells[0].text)
            hours.append('')
            salary.append(cells[1].text)
        else:
            print('WTF!')
       
    contract = to_string(contract)    
    hours = to_string(hours)   
    #salary = to_string(salary)   
    return contract, hours, salary

# Funcion para transformar el salario en un valor numerico
def parse_min_max_salary(salary):
    min_salary = []
    max_salary = []
    salary_rate = []
    for i in range(0, len(salary)):
        words = salary[i].split(" ")
        #print words
        # Loop and print each city name.
        rate = 'Sin determinar'
        min = 99999
        max = 0
        for w in words:
            if "/"  in w:
                rate = w
                
            n = str(re.sub(r"\D", "", w))
            if n.isdigit():
                n = int(n)
                if n < min:
                    min = n
                if n > max:
                    max = n

        if min == 99999:
            min = 0
        
        min_salary.append(min)
        max_salary.append(max)
        salary_rate.append(rate)
    
    salary_rate = to_string(salary_rate)
    return min_salary, max_salary, salary_rate

# Funcion para mostrar la informacion previa inclusion en un DataFrame
def print_jobs(title, organization, location, description, contract, hours, min_salary, max_salary, salary_rate):
    print 'Ofertas de trabajo\n'
    for i in range(0,len(title)):  
        print 'Oferta %d' % (i+1)
        print
        print 'Puesto: ' + title[i]
        print 'Organizacion: ' + organization[i]
        print 'Lugar: ' + location[i]
        print 'Descripcion: ' + description[i]
        print 'Tipo de Contrato: ' + contract[i]
        print 'Jornada: ' + hours[i]
        print 'Salario minimo: %d' % min_salary[i]
        print 'Salario maximo: %d' % max_salary[i]
        print 'Tipo de salario : ' + salary_rate[i]
        print
        
# Funcion para parsear una pagina
def page2pandas(page,r):
    page.encoding = ENCODING
    
    parser = etree.HTMLParser()
    tree   = etree.parse(StringIO(page.text), parser)

    #result = etree.tostring(tree.getroot(), pretty_print=True, method="html")
    #print(result)
    
    title = to_string(tree.xpath("//span[@itemprop='title']//text()"))
    organization = to_string(tree.xpath("//span[@itemprop='name']//text()"))
    location = to_string(tree.xpath("//span[@itemprop='jobLocation']//text()"))
    description = to_string(tree.xpath("//p[@itemprop='description']//text()"))

    details = tree.xpath("//ul[@class='tag-group hide-small-device']")
    contract, hours, salary = parse_details(details)
    min_salary, max_salary, salary_rate = parse_min_max_salary(salary)
    
    #print_jobs(title, organization, location, description, contract, hours, min_salary, max_salary, salary_rate)
    
    data = pd.DataFrame()
    data['title'] = title
    data['organization'] = organization
    data['location'] = location
    data['description'] = description
    data['contract'] = contract
    data['hours'] = hours
    data['min_salary'] = min_salary
    data['max_salary'] = max_salary
    data['salary_rate'] = salary_rate
    data['key'] = r
    return data

Programa

In [4]:
# Generamos un DataFrame que incluye los resultados de las palabras clave
roles = ['hadoop', 'spark', 'sql', 'python', 'java', 'scala', 'r']

pds = []
for r in roles:
    payload = {'inicio': 1, 'region' : 'local', 'palabra' : r, 'of_area' : 150, 'resultados': 10000}
    page = requests.post(URL, data=payload)
    pds.append(page2pandas(page,r))

Mostramos algunos de los datos recogidos

In [5]:
d = pd.concat(pds, axis=0)
d.head(10)

,title,organization,location,description,contract,hours,min_salary,max_salary,salary_rate,key
0,Desarrollo Hadoop - Big Data,TCP a UST Global Company,Madrid,En este momento precisamos reforzar el equipo ...,Contrato indefinido,Jornada completa,0,0,Sin determinar,hadoop
1,Perfil Big Data - Hadoop,CAST INFO,Madrid,"Cast Info, desde su nacimiento en 1.993, propo...",Contrato no especificado,Jornada completa,0,0,Sin determinar,hadoop
2,Desarrollador Python+ Hadoop,ALTEN TIC Madrid,Madrid,Desde el Departamento de Seleccion de ALTEN bu...,Contrato no especificado,Jornada completa,0,0,Sin determinar,hadoop
3,Ingeniero Big Data - Hadoop,Innovati,Madrid,Precisamos incorporar perfilles especializados...,Contrato indefinido,Jornada completa,0,0,Sin determinar,hadoop
4,Administrador Cloudera (hadoop)/Big Data/Cassa...,Sisnet Sistemas Netware,Madrid,"SISNET SISTEMAS NETWARE, consultora del sector...",Contrato indefinido,Jornada completa,30000,33000,Bruto/ano,hadoop
5,Analytics Architect,Ammeon,1 Million,Ammeon is a professional services company offe...,Contrato indefinido,Jornada completa,0,0,Sin determinar,hadoop
6,Programador Big Data Analytics,TCP a UST Global Company,Madrid,En este momento precisamos reforzar el equipo ...,Contrato indefinido,Jornada completa,0,0,Sin determinar,hadoop
7,Consultor Big Data,Sopra - Madrid,Madrid,"Sopra selecciona, para importante proyectos de...",Contrato indefinido,Jornada completa,0,0,Sin determinar,hadoop
8,Experto BigData,METRICA CONSULTING,Madrid,"Metrica Consulting, compania de servicios y so...",Contrato indefinido,Jornada completa,0,0,Sin determinar,hadoop
9,"DESARROLLADOR BIG DATA (SPARK,CASSANDRA)",StratioBD,Pozuelo De Alarcon,En Paradigma buscamos DESARROLLADORES BIG DATA...,Contrato indefinido,Jornada completa,36000,42000,Bruto/ano,hadoop


Mostramos algunas estadisticas basicas

In [6]:
print d.describe(include=['object']).transpose()
print
print d.describe().transpose()

             count unique                                                top  \
title         3201   2286                                   Programador Java   
organization  3201    974             everis Ofertas de empleo Profesionales   
location      3201    309                                             Madrid   
description   3201   2346  ZEMSANIA ICT OUTSOURCING SERVICES es una Multi...   
contract      3201      6                                Contrato indefinido   
hours         3201      8                                   Jornada completa   
salary_rate   3201      4                                     Sin determinar   
key           3201      6                                               java   

              freq  
title           21  
organization    72  
location      1338  
description     18  
contract      1740  
hours         3100  
salary_rate   2340  
key           1465  

            count         mean           std  min  25%  50%    75%    max
min_salary   32

## 2. NLTK

Programa

* Vamos a realizar el procesamiento NLP sobre el campo Titulo - Un campo de texto libre que describe la posicion de trabajo

* El campo no tiene limites en extension ni un lenguaje definido. Combina palabras propias - skills - con alguna estrutura semantica que identifique el rol

In [7]:
title = d['title']
title.head(10)

0                         Desarrollo Hadoop - Big Data
1                             Perfil Big Data - Hadoop
2                         Desarrollador Python+ Hadoop
3                          Ingeniero Big Data - Hadoop
4    Administrador Cloudera (hadoop)/Big Data/Cassa...
5                                  Analytics Architect
6                       Programador Big Data Analytics
7                                   Consultor Big Data
8                                     Experto  BigData
9             DESARROLLADOR BIG DATA (SPARK,CASSANDRA)
Name: title, dtype: object

### NLP basico

Librerias

In [8]:
from __future__ import division  # Python 2 users only
from collections import Counter

import string
import pprint
import re

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer

Programa

* Una primera aproximacion al procesamiento consiste en utilizar una bateria de transformaciones a nivel de palabra

* Mediante esta libreria realizamos:
    - Ponemos el texto en minusculas
    - Aplicamos un tokenizador basico para quedarnos con caracteres alfanumericos
    - Eliminamos Stopwords del Español, dado que las expresiones tienen su sintaxis en este idioma

In [9]:
def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = filter(lambda token: token not in set(stopwords.words('spanish')), tokens)
    return filtered_words

* Aplicamos esta funcion de filtro a todos los Titulos

In [10]:
l=[]
for index, row in d.iterrows():
    l.extend(preprocess(row['title']))

* Realizamos la agrupacion y conteo

In [11]:
print Counter(l).most_common(100)

[('programador', 1164), ('java', 895), ('analista', 677), ('j2ee', 290), ('net', 283), ('senior', 247), ('sql', 210), ('programadores', 203), ('tecnico', 196), ('desarrollador', 186), ('developer', 164), ('consultor', 158), ('junior', 158), ('data', 157), ('sistemas', 154), ('web', 151), ('analistas', 136), ('software', 132), ('oracle', 123), ('big', 117), ('administrador', 114), ('c', 103), ('ingeniero', 99), ('arquitecto', 99), ('engineer', 83), ('pl', 80), ('ingles', 72), ('desarrollo', 68), ('informatico', 68), ('php', 63), ('aplicaciones', 61), ('server', 61), ('as', 59), ('it', 55), ('bi', 52), ('spring', 51), ('proyecto', 50), ('end', 48), ('linux', 47), ('funcional', 41), ('analyst', 40), ('android', 40), ('front', 38), ('javascript', 38), ('alto', 37), ('soporte', 37), ('business', 37), ('beca', 36), ('architect', 36), ('mallorca', 36), ('sap', 35), ('datos', 35), ('ap', 34), ('palma', 33), ('microsoft', 31), ('qa', 31), ('backend', 31), ('jee', 30), ('sector', 30), ('manager'

* Conclusiones
    - Con esta tecnica obtenemos conteos sobre palabras significativas
    - Sin embargo, aparecen repetidas palabras parecidas (programador, programadores, ...)
    - Ademas, perdemos contexto sobre informacion util. Al no tratar semanticamente la expresion, no sabemos si hace falta un programador java o un analista java o igualmente solo sabriamos que hace falta un programador pero no su veterania o el lenguaje.
    - Finalmente, el hecho de combinar palabras en español y palabras en ingles, nos supone un gran problema
    - ESTA APROXIMACION NO NOS VALE

* Corolario
    - Podriamos solucionar el segundo problema mediante un Lematizador

* Definimos una funcion que nos permita lematizar una serie de tokens a partir de un Stemmer

In [12]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

* Aplicamos esta nueva funcion a todos los Titulos

In [13]:
stemmer = SnowballStemmer('spanish')
stemmed = stem_tokens(l, stemmer)

* Realizamos la agrupacion y conteo

In [14]:
count = Counter(stemmed)
print count.most_common(100)

[(u'program', 1375), (u'jav', 895), (u'anal', 813), (u'j2e', 291), (u'net', 283), (u'desarroll', 281), (u'senior', 247), (u'tecnic', 216), (u'sql', 210), (u'dat', 192), (u'consultor', 186), (u'develop', 164), (u'sistem', 158), (u'junior', 158), (u'web', 151), (u'softwar', 132), (u'oracl', 125), (u'administr', 124), (u'arquitect', 123), (u'big', 117), (u'ingenier', 113), (u'informat', 112), (u'c', 103), (u'engin', 86), (u'pl', 80), (u'proyect', 78), (u'ingles', 72), (u'php', 63), (u'serv', 62), (u'aplic', 61), (u'as', 59), (u'it', 55), (u'bi', 52), (u'spring', 51), (u'end', 48), (u'funcional', 47), (u'linux', 47), (u'test', 47), (u'andro', 40), (u'analyst', 40), (u'front', 38), (u'expert', 38), (u'javascript', 38), (u'alto', 37), (u'bec', 37), (u'soport', 37), (u'business', 37), (u'architect', 36), (u'mallorc', 36), (u'sap', 35), (u'jef', 34), (u'ap', 34), (u'palm', 33), (u'microsoft', 31), (u'qa', 31), (u'backend', 31), (u'jee', 30), (u'sector', 30), (u'segur', 29), (u'manag', 29), (u'

* Conclusiones
    - Con esta tecnica conseguimos un mayor nivel de agrupacion de palabras similares
    - Sin embargo, perdemos el significado de la palabra (anal...)
    - Ademas, no solucionamos ninguno de los demas problemas

### NLP Avanzado

Librerias

In [15]:
from nltk.corpus import wordnet
from nltk.corpus import stopwords

Programa

* Para la segunda aproximacion, parece obvio que necesitamos mantener cierta estructura semantica

* Para ello, he implementado una funcion mas avanzada que hace uso de tecnicas mas complejas para retener cierta estructura semantica. El codigo esta basado en el Paper de Su Nam Kim.
    - Construimos mediante una expresion regular una gramatica
    - Realizamos el tageado del Titulo y construimos un arbol semantico 
    - Aplicamos la gramatica al arbol
    - Normalizamos las palabras convirtiendolas a minuscula y aplicando un lematizador
    - Eliminamos las stopword españolas
    - Filtramos las palabras en español dado que los skills son en general palabras inglesas 

In [16]:
def process(text):
    sentence_re = r'''(?x)      # set flag to allow verbose regexps
          ([A-Z])(\.[A-Z])+\.?  # abbreviations, e.g. U.S.A.
        | \w+(-\w+)*            # words with optional internal hyphens
        | \$?\d+(\.\d+)?%?      # currency and percentages, e.g. $12.40, 82%
        | \.\.\.                # ellipsis
        | [][.,;"'?():-_`]      # these are separate tokens
    '''

    lemmatizer = nltk.WordNetLemmatizer()

    grammar = r"""
        NBAR:
            {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns

        NP:
            {<NBAR>}
            #{<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
    """
    chunker = nltk.RegexpParser(grammar)

    toks = nltk.regexp_tokenize(text, sentence_re)

    postoks = nltk.tag.pos_tag(toks)

    tree = chunker.parse(postoks)

    stopwords = nltk.corpus.stopwords.words('spanish')

    def leaves(tree):
        """Finds NP (nounphrase) leaf nodes of a chunk tree."""
        for subtree in tree.subtrees(filter = lambda t: t.label()=='NP'):
            yield subtree.leaves()

    def normalise(word):
        """Normalises words to lowercase and stems and lemmatizes it."""
        word = word.lower()
        word = lemmatizer.lemmatize(word)
        return word

    def acceptable_word(word):
        """Checks conditions for acceptable word: length, stopword."""
        accepted = bool(2 <= len(word) <= 40 and word.lower())
        
        if accepted:
            if word.lower() not in stopwords:
                if len(wordnet.synsets(word.lower())) > 0: 
                    return True
                else:
                    return False
            else:
                return False
        else:
            return False


    def get_terms(tree):
        for leaf in leaves(tree):
            term = [ normalise(w) for w,t in leaf if acceptable_word(w) ]
            yield term

    terms = get_terms(tree)
    try:
        first = terms.next()
    except StopIteration:
        first = ''
        
    return first 

* Aplicamos esta funcion de filtro a todos los Titulos
    - Nos quedamos solo con la primera expresion, ya que sera la mas significativa del rol y tendra el mayor valor semantico

In [17]:
l=[]
for index, row in d.iterrows():
    terms = process(row['title'])
    better_title = str(' '.join(terms))
    if better_title :
        l.append(better_title)

* Realizamos la agrupacion y conteo

In [18]:
print Counter(l).most_common(100)

[('java', 382), ('senior', 65), ('big data', 63), ('web', 52), ('junior', 35), ('senior java', 29), ('junior java', 28), ('oracle', 28), ('bi', 21), ('developer', 20), ('java oracle', 16), ('software', 14), ('asp', 14), ('java spring', 14), ('software developer', 12), ('data scientist', 12), ('android', 12), ('java developer', 12), ('server', 12), ('dynamic', 10), ('engineer', 10), ('java junior', 10), ('software engineer', 10), ('senior developer', 9), ('tester', 9), ('sa', 9), ('big data engineer', 9), ('big data architect', 8), ('java swing', 8), ('senior java developer', 8), ('spring', 8), ('sr java', 8), ('tic', 8), ('visual basic', 7), ('sap basis', 7), ('python', 7), ('base', 7), ('business intelligence', 7), ('sap', 7), ('helpdesk', 7), ('delphi', 7), ('cobol', 7), ('dba oracle', 7), ('net developer', 6), ('java senior', 6), ('window', 6), ('project manager big data', 6), ('test analyst', 6), ('bpm', 6), ('alto', 5), ('sr', 5), ('dba', 5), ('senior architect', 5), ('java web', 

* Conclusiones
    - Con este procesamiento avanzado podemos ver que para el idioma ingles, se recogen con mayor detalle la descripcion del puesto (big data, senior java developer, project manager big data)
    - Con palabras sueltas, seguimos perdiendo el contexto. Quizas podriamos mejorar si combinamos para una misma oferta varias expresiones semanticas si estas solo tienen una palabra
    - Podria ayudar el tratar el idioma español de forma analoga y combinarlo con el ingles
    - Muchos Titulos quedan sin transformacion; combinar 2 idiomas es un obstaculo para NLP
    - En el caso particular del ingles, esta tecnica mejora bastante las tecnicas simples de procesamiento por palabra
    - RETENER SIGNIFICADO SEMANTICO ES ESENCIAL

* Nota al profesor:
    - Con este trabajo he buscado enfrentarme a un problema real y combinar tecnicas para hallar una solucion
    - Si bien hay muchos ejemplos de aplicacion de NLP que funcionan a las mil maravillas, he considerado mas interesante atacar un problema real
    - Sin embargo, la fria realidad es que - con los conocimientos que tengo - atacar NLP multilenguaje y estructuras semanticas complejas NO ES FACIL
    - Espero que se valore el trabajo y la busqueda de soluciones vs el caso de libro fantastico

Gracias!